<a href="https://colab.research.google.com/github/dggasque/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
#Map Interest to numerical value
df['interest_numerical'] = df['interest_level'].map({'low':1, 'medium':2, 'high':3})

In [0]:
# create feature that counts the length of the description
df['description_length'] = df['description'].apply(lambda x: len(str(x)))

In [0]:
# Create allows cats and dogs column

df['cats_and_dogs_allowed'] = df['cats_allowed'] + df['dogs_allowed']
df['cats_and_dogs_allowed'] = df['cats_and_dogs_allowed'].map({2:1, 1:0, 0:0})

In [148]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,interest_numerical,description_length,cats_and_dogs_allowed
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,588,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8,1
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,691,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,492,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,479,0


In [0]:
# Convert created feature from obect to date time
df['created'] = pd.to_datetime(df['created'])

In [0]:
# Split data into train and test datasets by month 
train = df[df['created'].dt.month < 6]
test = df[df['created'].dt.month >= 6]

In [160]:
train.shape

(31844, 37)

In [161]:
test.shape

(16973, 37)

In [162]:
train.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,interest_numerical,description_length,cats_and_dogs_allowed
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,691,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,492,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,479,0
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,0
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,579,1


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [169]:
# Set baseline 
target = 'price'
y_train = train[target]
y_test = test[target]

# Get mean baseline
print('Mean Baseline (using 0 features)')
guess = y_train.mean()

# Train Error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: ', mae)

# Test Error
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ', mae)


Mean Baseline (using 0 features)
Train Error:  1201.8811133682555
Test Error:  1197.7088871089013


In [0]:
#Initiate model
model = LinearRegression()

In [156]:
# Arrange X feature matrices and y target vectors
features = ['bathrooms', 'bedrooms', 'interest_numerical', 
            'elevator', 'doorman', 'garden_patio', 'cats_and_dogs_allowed']

target = 'price'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print(f'Train Root Mean Squared Error: ', rmse)
print(f'Train Absolute Error: ', mae)
print(f'Train R^2 Score: ', r2)

# Apply the model to test data
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Test Root Mean Squared Error: ', rmse)
print('Test Absolute Error: ', mae)
print('Test R^2 Score: ', r2)

Train Root Mean Squared Error:  1142.7509003718446
Train Absolute Error:  743.4873579258164
Train R^2 Score:  0.5794306754521089
Test Root Mean Squared Error:  1120.7748964365571
Test Absolute Error:  737.9439808227914
Test R^2 Score:  0.5958404849171826


In [157]:
model.coef_

array([1800.39216044,  473.21299696, -456.80179696,  153.90726859,
        598.34284915,  137.45733493,   75.41821153])

In [158]:
model.intercept_

938.687920738811

Change Features to see if we can improve model.

In [81]:
# Arrange X feature matrices and y target vectors
features = ['bathrooms', 'bedrooms', 
            'elevator', 'doorman', 
            'garden_patio', 'dishwasher',
            'cats_and_dogs_allowed',
            'hardwood_floors', 'laundry_in_unit', 
            'interest_numerical']

target = 'price'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print(f'Train Root Mean Squared Error: ', rmse)
print(f'Train Absolute Error: ', mae)
print(f'Train R^2 Score: ', r2)

# Apply the model to test data
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Test Root Mean Squared Error: ', rmse)
print('Test Absolute Error: ', mae)
print('Test R^2 Score: ', r2)

Train Root Mean Squared Error:  1126.6224676084075
Train Absolute Error:  731.5900163650653
Train R^2 Score:  0.5912184703486918
Test Root Mean Squared Error:  1111.086979421348
Test Absolute Error:  730.2458625292842
Test R^2 Score:  0.602797352576476


In [89]:
# Arrange X feature matrices and y target vectors
features = ['bathrooms', 'bedrooms', 
            'elevator', 'doorman', 
            'garden_patio', 'dishwasher',
            'cats_and_dogs_allowed',
            'hardwood_floors', 'laundry_in_unit', 
            'interest_numerical', 'fitness_center',
            'high_speed_internet', 'balcony']

target = 'price'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print(f'Train Root Mean Squared Error: ', rmse)
print(f'Train Absolute Error: ', mae)
print(f'Train R^2 Score: ', r2)

# Apply the model to test data
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Test Root Mean Squared Error: ', rmse)
print('Test Absolute Error: ', mae)
print('Test R^2 Score: ', r2)

Train Root Mean Squared Error:  1122.8988234911337
Train Absolute Error:  727.9979041612262
Train R^2 Score:  0.5939161645834881
Test Root Mean Squared Error:  1107.2310022755505
Test Absolute Error:  727.7098129638051
Test R^2 Score:  0.605549516317442


In [125]:
# Arrange X feature matrices and y target vectors
features = ['bathrooms', 'bedrooms', 
            'elevator', 'doorman', 
            'garden_patio', 'dishwasher',
            'cats_and_dogs_allowed',
            'hardwood_floors', 'laundry_in_unit', 
            'interest_numerical', 'fitness_center',
            'high_speed_internet', 'balcony',
            'exclusive', 'new_construction',
            'pre-war', 'longitude',
            'outdoor_space', 'common_outdoor_space']

target = 'price'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print(f'Train Root Mean Squared Error: ', rmse)
print(f'Train Absolute Error: ', mae)
print(f'Train R^2 Score: ', r2)

# Apply the model to test data
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Test Root Mean Squared Error: ', rmse)
print('Test Absolute Error: ', mae)
print('Test R^2 Score: ', r2)

Train Root Mean Squared Error:  1068.6520678096726
Train Absolute Error:  671.240559617067
Train R^2 Score:  0.6322039086622272
Test Root Mean Squared Error:  1054.2945696443633
Test Absolute Error:  675.7410233997266
Test R^2 Score:  0.642365047151176


In [134]:
# Arrange X feature matrices and y target vectors
features = ['bathrooms', 'bedrooms', 
            'elevator', 'doorman', 
            'garden_patio', 'dishwasher',
            'cats_and_dogs_allowed',
            'hardwood_floors', 'laundry_in_unit', 
            'interest_numerical', 'fitness_center',
            'high_speed_internet', 'balcony',
            'exclusive', 'new_construction',
            'pre-war', 'longitude',
            'outdoor_space', 'common_outdoor_space',
            'laundry_in_building', 'wheelchair_access']

target = 'price'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print(f'Train Root Mean Squared Error: ', rmse)
print(f'Train Mean Absolute Error: ', mae)
print(f'Train R^2 Score: ', r2)

# Apply the model to test data
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Test Root Mean Squared Error: ', rmse)
print('Test Mean Absolute Error: ', mae)
print('Test R^2 Score: ', r2)

Train Root Mean Squared Error:  1068.2281463939723
Train Mean Absolute Error:  670.8666911549153
Train R^2 Score:  0.6324956513524763
Test Root Mean Squared Error:  1053.6289182867747
Test Mean Absolute Error:  675.1188979548431
Test R^2 Score:  0.6428165054941248


In [165]:
# Arrange X feature matrices and y target vectors
features = ['bathrooms', 'bedrooms', 
            'elevator', 'doorman', 
            'garden_patio', 'dishwasher',
            'cats_and_dogs_allowed',
            'hardwood_floors', 'laundry_in_unit', 
            'interest_numerical', 'fitness_center',
            'high_speed_internet', 'balcony',
            'exclusive', 'new_construction',
            'pre-war', 'longitude',
            'outdoor_space', 'common_outdoor_space',
            'laundry_in_building', 'wheelchair_access',
            'roof_deck']

target = 'price'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print(f'Train Root Mean Squared Error: ', rmse)
print(f'Train Mean Absolute Error: ', mae)
print(f'Train R^2 Score: ', r2)

# Apply the model to test data
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Test Root Mean Squared Error: ', rmse)
print('Test Mean Absolute Error: ', mae)
print('Test R^2 Score: ', r2)

Train Root Mean Squared Error:  1067.2379546649174
Train Mean Absolute Error:  669.9604176894602
Train R^2 Score:  0.6331766502750503
Test Root Mean Squared Error:  1052.5360695516476
Test Mean Absolute Error:  674.3866557421618
Test R^2 Score:  0.6435570794942888


Best model has test error of $674.39
features = ['bathrooms', 'bedrooms', 
            'elevator', 'doorman', 
            'garden_patio', 'dishwasher',
            'cats_and_dogs_allowed',
            'hardwood_floors', 'laundry_in_unit', 
            'interest_numerical', 'fitness_center',
            'high_speed_internet', 'balcony',
            'exclusive', 'new_construction',
            'pre-war', 'longitude',
            'outdoor_space', 'common_outdoor_space',
            'laundry_in_building', 'wheelchair_access',
            'roof_deck']